In [127]:
import numpy as np

initial_chord = [60, 64, 67, 71] #Defines by midi number the notes of the chord given by the user

upper_time_signature = 4 #Indicates how many beats are grouped together in a measure(bar), value given by the user
lower_time_signature = 4 #Indicates the note value that represents one beat, value given by the user 

#Silulating a melody created by the algorithm, p = picth and r = rhythm
p = [60, 62, 64, 65, 67, 69, 71, 72, 74, 76, 77]
r = [0.5, 0.5, 0.5, 0.5, 1.5, 0.5, 0.5, 0.5, 0.5, 0.5, 2.0]

N = len(p)
L = sum(r)

M = int(L / upper_time_signature) #M defines the number of measures in the melody

if lower_time_signature == 2:
    beat = 2
elif lower_time_signature == 4:
    beat = 1
elif lower_time_signature == 8:
    beat = 0.5 

print(N, L, M)#teste

11 8.0 2


In [128]:
#Here start the equation to define the fitness function on stability

#Cn returns 1 when the note is a chord note, else 0
#Eq.(4)
def C(p):
    return 1 if p in initial_chord else 0

#CT calculates a fraction of CHORD TONES
#Eq.(3)
def CT(p, r):
    list_CT = [] 
    for i in range(N):
        num = C(p[i]) * r[i]
        den = L
        list_CT.append(num/den)
    return sum(list_CT)


print(CT(p, r))#teste

0.375


In [129]:
#Eq.(6)
def U(x): 
    return 1 if x>=0 else 0
    
#SM calculates te amount of step motion(midi value less than 2) in p
#Asigns a higher score in the range of [0.0, 1.0] to the melody that contains more step motions. 
#Eq.(5)
def SM(p):
    list_SM = []
    for i in range(N-1):
        num = U(2 - abs(p[i] - p[i+1]))
        den = N-1
        list_SM.append(num/den)
    return sum(list_SM)

print(SM(p))#teste

0.9999999999999999


In [130]:
#DL evaluates a fraction if three consecutive notes in descending lines of the pitch sequence p
#Returns 1 only if the three successive notes are descending    
#Eq.(7)
def DL(p):
    list_DL = []
    for i in range(N-2):
        num = U(p[i] - p[i+1] - 1) * U(p[i+1] - p[i+2] - 1)
        den = N-2
        list_DL.append(num/den)
    return sum(list_DL)

print(DL(p))#teste

0.0


In [131]:
#fitness_1 calculates which is the higher stability value and assigns it with alpha, beta, gamma
#A higher fitness value presents a better melody in terms of stability
#Eq.(2)
def fitness_1(p, r, alpha=3.0, beta=2.0, gamma=1.0, delta=100.0):
    list_fitness_1 = []
    list_fitness_1.append(CT(p,r))
    list_fitness_1.append(SM(p))
    list_fitness_1.append(DL(p))
    fitness_1_sorted = sorted(list_fitness_1, reverse = True)
    print (fitness_1_sorted) #teste para ver a lista ordenada de maior para menor
    
    more_stability = fitness_1_sorted[0]
    midle_stability = fitness_1_sorted[1]
    less_stability = fitness_1_sorted[2]
    
    return ((alpha*more_stability) + (beta*midle_stability) + (gamma*less_stability)) - (delta*P(p, r))
       
print (fitness_1(p, r, alpha=3.0, beta=2.0, gamma=1.0, delta=100.0))#teste

[0.9999999999999999, 0.375, 0.0]
-3196.25


In [132]:
#Here start the equation to define the fitness function on tension

#NT computes a fraction of non-chord tone(notes that is not in initial_chord) of the melody
#Eq.(9)
def NT(p, r):
    return 1.0 - CT(p, r)

print (NT(p, r))#teste

0.625


In [133]:
#LM measures a ratio of skip motion(notes that are connected by a midi value higher than 2)
#Returns 1 when two consecutive notes are connected by a skip motion
#Thus, LM(p) returns a higher value when the pitch vector p is comprised of more skip motions
#Eq.(10)
def LM(p):
    list_LM = []
    for i in range(N-1):
        num = U(abs(p[i] - p[i+1]) - 2)
        den = N-1
        list_LM.append(num/den)
    return sum(list_LM)

print (LM(p))#teste

0.7


In [134]:
#AL evaluates a fraction of three successive notes in ascending lines (in contrast with DL(p))
#Eq.(11)
def AL(p):
    list_AL = []
    for i in range(N-2):
        num = U(p[i+2] - p[i+1] - 1) * U(p[i+1] - p[i] - 1)
        den = N-2
        list_AL.append(num/den)
    return sum(list_AL)

print (AL(p))#teste

1.0000000000000002


In [135]:
#fitness_2 calculates which is the higher tension value and assigns it with alpha, beta, gamma
#A higher fitness value presents more instability on the melody
#Eq.(8)
def fitness_2(p, r, alpha=3.0, beta=2.0, gamma=1.0, delta=100.0):
    list_fitness_2 = []
    list_fitness_2.append(NT(p,r))
    list_fitness_2.append(LM(p))
    list_fitness_2.append(AL(p))
    fitness_2_sorted = sorted(list_fitness_2, reverse = True)
    print (fitness_2_sorted)#teste para ver a lista ordenada de maior para menor
    
    more_tension = fitness_2_sorted[0]
    midle_tension = fitness_2_sorted[1]
    less_tension = fitness_2_sorted[2]
    
    return ((alpha*more_tension) + (beta*midle_tension) + (gamma*less_tension)) - (delta*P(p, r)) 
    
       
print (fitness_2(p, r, alpha=3.0, beta=2.0, gamma=1.0, delta=100.0))#teste

[1.0000000000000002, 0.7, 0.625]
-3194.975


In [136]:
#This function defines and separates the measures in the melody

def divide(rhythm_list, pitch_list, target):
    # vamos começar a contagem da soma em zero
    count = 0
    count_beat = 0
   
    # aqui está a nossa lista com as "fatias" que queremos
    full_melody_list = []
    full_pitch_list = []
    S_index = []
   
    # aqui fica uma measure
    measure = []
    measure_pitch = []
    for i in range(len(rhythm_list)):
        item = rhythm_list[i]
        count += item # aqui somamos o nosso contador
        measure.append(item) # adicionamos o item atual na lista da measure
        measure_pitch.append(pitch_list[i])
       
        # quando o contador bate o target "fecharemos" o measure atual
        # e prepararemos um novo measure e resetaremos o contador
        if count >= target:
            full_melody_list.append(measure)
            full_pitch_list.append(measure_pitch)
            measure = []
            measure_pitch = []
            count = 0
           
    return full_melody_list, full_pitch_list

full_melody_list, full_pitch_list = divide(r, p, upper_time_signature)
    
print(full_melody_list)#teste
print(full_pitch_list)#teste

[[0.5, 0.5, 0.5, 0.5, 1.5, 0.5], [0.5, 0.5, 0.5, 0.5, 2.0]]
[[60, 62, 64, 65, 67, 69], [71, 72, 74, 76, 77]]


In [137]:
def get_s_index(full_melody_list, full_pitch_list):
    number_measures = len(full_melody_list)
    
    # processando cada measure individualmente
    current_lower_index = 0
    S_index = np.array([])
    
    for i in range(number_measures):
        current_pitch = np.array(full_pitch_list[i])
        current_melody = np.array(full_melody_list[i])
        
        # 1a regra: soma dos primeiros rhythms não-negativos que alcancem o valor do beat
        # pegando somente o índice dos pitches positivos
        positive_pitch_indexes = np.argwhere(np.array(full_pitch_list[i]) > 0).flatten()
        
        # somando os rhythms desta measure até dar o beat
        soma = np.cumsum(current_melody[positive_pitch_indexes])
        threshold = np.argwhere(soma >= beat)[0][0]
        S_index = np.append(np.array(range(threshold+1)) + current_lower_index, S_index)
        
        # 2a regra: considerar upper time signature
        beats, _ = divide(full_melody_list[i], full_pitch_list[i], beat)
        S_index = np.append(np.array(range(len(beats[0]))) + current_lower_index, S_index)
        
        if upper_time_signature == 4 or upper_time_signature == 6:
            previous_beats = len(np.array(beats[:int(upper_time_signature/2)]).flatten())
            len_current_beat = len(np.array(beats[int(upper_time_signature/2)]).flatten())
            S_index = np.append(np.array(range(previous_beats, previous_beats + len_current_beat)) +
                  current_lower_index, S_index)
        current_lower_index += len(current_pitch)
        
    return np.unique(S_index).astype(int)

S = get_s_index(full_melody_list, full_pitch_list)

In [138]:
#Defining the rules on the plausible pleasant non-chord tones

#Rule 1: Non-chord tone on the strong beat should move to a chord tone by a step motion

#Eq.(14)
def Nn(p): 
    return 1 if p not in initial_chord else 0
    
#UT measures the number of unresolved non-chord tones on the strong beats
#S is a set of indices of the strong beats 
#Eq.(13)
def UT(p):
    list_UT = []
    for i in S:
        num = Nn(p[i])
        if i != len(p) -1:
            num *= (Nn(p[i+1]) + U(abs(p[i] - p[i+1])) - 2)
        list_UT.append(num)
    return sum(list_UT)
        
print (UT(p))#teste

0


In [139]:
#Rule 2: All non-chord tones should be connected with at least one adjacent chord tone by a step motion

#Eq.(15)
def NCC(p):
    list_NCC = []
    for i in range(N-1): #correction in the equation
        num = (Nn(p[i])) * (Nn(p[i-1]) + U(abs(p[i-1] - p[i]) - 2)) * (Nn(p[i+1]) + U(abs(p[i] - p[i+1]) - 2))
        list_NCC.append(num)
    return sum(list_NCC)


print (NCC(p))#teste

8


In [140]:
#Rule 3: Skip motion on the non-chord tones should not be over the third interval

#NCL Captures the number of intervals which are greater than the third with an adjacent non-chord tone
#Eq.(16)
def NCL(p):
    list_NCL = []
    for i in range(N-1): #correction in the equation
        num = Nn(p[i]) * (U(4 - abs(p[i-1] - p[i])) + U(4 - abs(p[i] - p[i+1])))
        list_NCL.append(num)
    return sum(list_NCL)

print (NCL(p))#teste

12


In [141]:
#Here start the defining the penalty function

#Minimizing MNC(p) can resolve a misused non-chord tones in the melody
#Eq.(17)
def MNC(p):
    return UT(p) + NCC(p) + NCL(p)
    
print (MNC(p))#teste

20


In [142]:
#AN investigate the number o avoid notes over the strong beats
#If both Nn(p[i]) and C(p[i] - 1) return 1, p[i] becomes a avoid note
#Eq.(18)
def AN(p):
    list_AN = []
    for i in S:
        num = Nn(p[i]) * C(p[i] - 1)
        list_AN.append(num)
    return sum(list_AN)

print (AN(p))#teste

1


In [143]:
#ON counts the number of measures(i.e., bar) that contains more than 50% of non-chord tones

#Eq.(19)

D = 10
        
def ON2(p, r):
    list_ON2 = []
    for j in range(len(p)):
        num = r[j] * C(p[j])
        list_ON2.append(num/sum(r))
    return sum(list_ON2)

def ON(full_pitch_list, full_melody_list):
    list_ON = []
    for i in range(M):
        num = U(0.5 - ON2(full_pitch_list[i], full_melody_list[i]))
        list_ON.append(num)
    return sum(list_ON)
        

print(ON2(p, r))#teste #Resultado da somatória dentro da Eq.(19)
print(ON(full_pitch_list, full_melody_list))#teste #O resultado final é este

0.375
1


In [144]:
#BL(p) computes the number of over-leap intervals
#Eq.(20)
def BL(p):
    list_BL = []
    for i in range(N-1):
        num = U(16 - abs(p[i] - p[i+1]))
        list_BL.append(num)
    return sum(list_BL)

print (BL(p))#teste

10


In [145]:
#Final equation to calculate the penalty

#Eq.(12)
def P(p, r):
    return ((MNC(p)) + (AN(p)) + (ON(full_pitch_list, full_melody_list)) + (BL(p))) 
    
print(P(p, r))#teste

32
